# 🌐 OpenAI Tool API

#### Weather, Geocoding, and Routing

### 1. 📦 Setup and Imports

In [ ]:
# Cell 1: Imports and Configuration

from __future__ import annotations
import os
import json
import requests
from typing import Any, Dict, Optional

from openai import OpenAI

MODEL = os.environ.get("OPENAI_MODEL", "gpt-4.1-mini")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY")


### 2. 🗺️ Google Maps Static API Visualization

In [ ]:
# Cell 2: Google Maps Static API (show marker or route)

def google_maps_static_map(
    lat: float, lon: float, zoom: int = 12, size: str = "600x400", label: str = "A"
) -> None:
    """
    Show a Google Maps static map with a marker at the specified coordinates.
    """
    url = (
        f"https://maps.googleapis.com/maps/api/staticmap?"
        f"center={lat},{lon}&zoom={zoom}&size={size}"
        f"&markers=color:red%7Clabel:{label}%7C{lat},{lon}"
        f"&key={GOOGLE_MAPS_API_KEY}"
    )
    display(Image(url=url))

def google_maps_route_static(
    origin_lat: float, origin_lon: float,
    dest_lat: float, dest_lon: float,
    zoom: int = 7, size: str = "600x400"
) -> None:
    """
    Show a static Google Map with a route (polyline) from origin to destination.
    """
    # Google Maps API expects origin/destination as strings
    url = (
        f"https://maps.googleapis.com/maps/api/staticmap?"
        f"size={size}&zoom={zoom}"
        f"&markers=color:blue%7Clabel:O%7C{origin_lat},{origin_lon}"
        f"&markers=color:green%7Clabel:D%7C{dest_lat},{dest_lon}"
        f"&path=color:0x0000ff|weight:5|{origin_lat},{origin_lon}|{dest_lat},{dest_lon}"
        f"&key={GOOGLE_MAPS_API_KEY}"
    )
    display(Image(url=url))


### 2. 🛠️ Tool Definitions

- Each tool is a standalone OpenAI-compatible JSON schema (strictly one per call).

- Weather Schema

In [ ]:
# Weather Tool (Open-Meteo or other weather API)
WEATHER_TOOL = {
    "type": "function",
    "name": "get_forecast",
    "description": "Return weather forecast for a lat/lon point.",
    "parameters": {
        "type": "object",
        "properties": {
            "lat": {"type": "number", "description": "Latitude (decimal degrees)."},
            "lon": {"type": "number", "description": "Longitude (decimal degrees)."},
            "hours": {
                "type": "integer",
                "minimum": 1,
                "maximum": 72,
                "default": 24,
                "description": "Forecast window in hours (1–72)."
            },
            "units": {
                "type": "string",
                "enum": ["imperial", "metric"],
                "default": "imperial",
                "description": "Unit system: 'imperial' (°F, mph), 'metric' (°C, km/h)."
            },
        },
        "required": ["lat", "lon"],
        "additionalProperties": False,
    },
    "strict": True,
}



- Geocoding Schema

In [ ]:
# Geocode Tool (Nominatim or other geocoder)
GEOCODE_TOOL = {
    "type": "function",
    "name": "geocode",
    "description": "Convert a place/address to latitude/longitude coordinates.",
    "parameters": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "User-supplied place or address (street, city, etc)."
            }
        },
        "required": ["address"],
        "additionalProperties": False,
    },
    "strict": True,
}


- Routing Schema

In [ ]:
# Route Tool (OSRM or other routing engine)
ROUTE_TOOL = {
    "type": "function",
    "name": "get_route",
    "description": (
        "Return driving directions between two coordinates. "
        "Use for route planning between origin and destination."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "origin_lat": {"type": "number", "description": "Origin latitude."},
            "origin_lon": {"type": "number", "description": "Origin longitude."},
            "dest_lat": {"type": "number", "description": "Destination latitude."},
            "dest_lon": {"type": "number", "description": "Destination longitude."},
        },
        "required": ["origin_lat", "origin_lon", "dest_lat", "dest_lon"],
        "additionalProperties": False,
    },
    "strict": True,
}


### 3. Geocoding 

- Real geocoding (address → lat/lon) via Nominatim

In [ ]:
# Cell 3: Geocode using Nominatim
def geocode(address: str) -> Dict[str, Any]:
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": address, "format": "json", "limit": 1}
    headers = {"User-Agent": "OpenAI-Tools-Example/1.0"}
    resp = requests.get(url, params=params, headers=headers, timeout=10)
    data = resp.json()
    if not data:
        return {"lat": None, "lon": None, "match": "No match"}
    top = data[0]
    return {
        "lat": float(top["lat"]),
        "lon": float(top["lon"]),
        "match": top.get("display_name", address),
    }


### 4. 🌤️ Weather with Open-Meteo

In [ ]:
# Cell 4: Weather for a coordinate (Open-Meteo)
def get_forecast(lat: float, lon: float, hours: int = 6, units: str = "imperial") -> Dict[str, Any]:
    url = "https://api.open-meteo.com/v1/forecast"
    temp_unit = "fahrenheit" if units == "imperial" else "celsius"
    wind_unit = "mph" if units == "imperial" else "kmh"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m,precipitation_probability,windspeed_10m",
        "temperature_unit": temp_unit,
        "windspeed_unit": wind_unit,
        "forecast_days": 1,
        "timezone": "auto",
    }
    resp = requests.get(url, params=params, timeout=10)
    data = resp.json()
    if "hourly" not in data:
        return {"error": "No forecast data found."}
    h = data["hourly"]
    return {
        "location": {"lat": lat, "lon": lon},
        "units": units,
        "forecast": [
            {
                "time": h["time"][i],
                "temp": h["temperature_2m"][i],
                "wind": h["windspeed_10m"][i],
                "precip_prob": h["precipitation_probability"][i],
            }
            for i in range(min(hours, len(h["time"])))
        ],
        "provider": "open-meteo.com"
    }


### 5. 🚗 Routing with OSRM

In [ ]:
# Cell 5: Driving route (OSRM)
def get_route(
    origin_lat: float, origin_lon: float, dest_lat: float, dest_lon: float
) -> Dict[str, Any]:
    url = (
        f"http://router.project-osrm.org/route/v1/driving/"
        f"{origin_lon},{origin_lat};{dest_lon},{dest_lat}"
    )
    params = {"overview": "false", "alternatives": "false", "steps": "true"}
    resp = requests.get(url, params=params, timeout=10)
    data = resp.json()
    if not data.get("routes"):
        return {"error": "No route found."}
    r = data["routes"][0]
    steps = []
    for leg in r["legs"]:
        for step in leg["steps"]:
            steps.append({
                "instruction": step["maneuver"]["instruction"],
                "distance_m": step["distance"],
                "duration_s": step["duration"]
            })
    return {
        "origin": {"lat": origin_lat, "lon": origin_lon},
        "destination": {"lat": dest_lat, "lon": dest_lon},
        "distance_km": r["distance"] / 1000,
        "duration_min": r["duration"] / 60,
        "steps": steps[:8],  # first 8 for demo
        "provider": "project-osrm.org"
    }


### 6. 🏙️ Demo: End-to-End with Google Maps

- (A) Geocode locations:

In [ ]:
# Cell 6a: Geocode two places
origin_addr = "New York, NY"
dest_addr = "Washington, DC"
origin_geo = geocode(origin_addr)
dest_geo = geocode(dest_addr)
print("Origin:", origin_geo)
print("Destination:", dest_geo)


- (B) Show both locations on Google Maps:

In [ ]:
# Cell 6b: Show each city as a marker
google_maps_static_map(origin_geo["lat"], origin_geo["lon"], label="O")
google_maps_static_map(dest_geo["lat"], dest_geo["lon"], label="D")


- (C) Show the driving route between the two cities:

In [ ]:
# Cell 6c: Show the route as a polyline on the map
google_maps_route_static(
    origin_lat=origin_geo["lat"], origin_lon=origin_geo["lon"],
    dest_lat=dest_geo["lat"], dest_lon=dest_geo["lon"]
)


- (D) Get weather for each city:

In [ ]:
# Cell 6d: Weather in each city
weather_origin = get_forecast(origin_geo["lat"], origin_geo["lon"])
weather_dest = get_forecast(dest_geo["lat"], dest_geo["lon"])
print("NYC Weather:", weather_origin["forecast"][:3])
print("DC Weather:", weather_dest["forecast"][:3])


- (E) Get route summary and steps:

In [ ]:
# Cell 6e: Get driving directions
route = get_route(
    origin_geo["lat"], origin_geo["lon"],
    dest_geo["lat"], dest_geo["lon"]
)
print(f"Route: {route['distance_km']:.1f} km, {route['duration_min']:.1f} min")
for s in route["steps"]:
    print("-", s["instruction"])


### 7. 🔄 Tool-Calling Helpers

In [ ]:
def _coerce_args( args: Any ) -> Dict[str, Any]:
    """
    Purpose:
        Convert the tool argument payload to a dictionary.
        (Handles cases where arguments may be dicts or JSON strings.)

    Parameters:
        args:
            The arguments object or string from the function call event.

    Returns:
        A dictionary of arguments (for use as Python function kwargs).
    """
    if isinstance(args, dict):
        return args
    if isinstance(args, str):
        return json.loads(args)
    try:
        return json.loads(str(args))
    except Exception as exc:
        raise ValueError(f"Unsupported arguments payload: {type(args)!r}") from exc


def _first_function_call(resp: Any) -> Optional[Any]:
    """
    Purpose:
        Locate the first function/tool call in the OpenAI API response.

    Parameters:
        resp:
            The object returned by client.responses.create(...) (OpenAI Responses API).

    Returns:
        The first output item with type 'function_call', or None if not present.
    """
    try:
        for item in resp.output:  # 'output' is a list of response events/messages
            if getattr(item, "type", None) == "function_call":
                return item
    except Exception:
        pass
    return None


### 8. 🏙️ Address → Weather → Routing

- (A) Geocode each address:

In [ ]:
# Cell 7a: Geocode both origin and destination
origin_addr = "New York, NY"
dest_addr = "Washington, DC"
origin_geo = geocode(origin_addr)
dest_geo = geocode(dest_addr)
print("Origin:", origin_geo)
print("Destination:", dest_geo)



- (B) Get weather for each city:

In [ ]:
# Cell 7b: Weather at origin
weather_origin = get_forecast(origin_geo["lat"], origin_geo["lon"], hours=6, units="imperial")
print("Weather in New York (next 6h):", weather_origin["forecast"])


In [ ]:
# Cell 7c: Weather at destination
weather_dest = get_forecast(dest_geo["lat"], dest_geo["lon"], hours=6, units="imperial")
print("Weather in Washington (next 6h):", weather_dest["forecast"])


- (C) Get route between them:

In [ ]:
# Cell 7d: Routing
route_result = get_route(
    origin_geo["lat"], origin_geo["lon"], dest_geo["lat"], dest_geo["lon"]
)
print("Route summary:", route_result["distance_km"], "km,", route_result["duration_min"], "min")
for s in route_result["steps"]:
    print("-", s["instruction"])
